In [ ]:

import os
import tensorflow as tf
import numpy as np
import json
import shutil
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


bs = 16
num_classes=3

img_h = 224
img_w = 224


# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
path = cwd + '/Dataset'
if os.path.exists(path):
  shutil.rmtree(path)

os.makedirs(path)


path = cwd + '/Dataset/test'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/training'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/training/no_one'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/training/someone'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/training/everyone'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/validation'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/validation/no_one'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/validation/someone'
if not os.path.exists(path):
    os.makedirs(path)

path = cwd + '/Dataset/validation/everyone'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
with open('../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/train_gt.json') as file:
    data = json.load(file)


    
    
#we put the images from MaskDataset to Dataset.
#and organize them in subdirectories. There is a subdirectory for each class
 

#training set
source = '../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/training'
dest = cwd + '/Dataset/training'

for filename in os.listdir(source):
  if data[filename] == 0:
    a=shutil.copy(source + '/' + filename,  dest + '/no_one/' + filename)

  elif data[filename] == 1:
    a=shutil.copy(source + '/' + filename, dest + '/everyone/' + filename)

  else:
    a=shutil.copy(source + '/' + filename, dest + '/someone/' + filename)


#test set
source =  '../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/test'
dest = cwd + '/Dataset/test'

for filename in os.listdir(source):
    a=shutil.copy(source + '/' + filename,  dest + '/' + filename)

In [ ]:
#Create validation set 20/80.

#Per ora non è randomico, ma sempre lo stesso a dogni run. così da ripetere gli esperimenti
#forse dovremmo farlo randomico? in tal caso basta fare uno shuffle ogni volta, prima di creare validation.

#se si runna più volte questa cella, un altro 20% degli elementi restanti nel training, vanno nel validation.
#se si vuole ricreare il validation, ri runnare prima la cella sopra.
#se si vuole svuotare e ricreare tutto il Datset, ri runnare da 2 celle sopra

source = cwd + '/Dataset/training'
dest = cwd + '/Dataset/validation'


#metto un contatore al 20% della dimensione di ogni classe del training. 
#fino a che non è 0, sposto dal training al validation.
count = len(os.listdir(source + '/no_one')) * 0.2
for filename in os.listdir(source + '/no_one'):
    if count > 0:
        count -= 1
        shutil.move(source + '/no_one/' + filename, dest + '/no_one/' + filename)
        
        
        
count = len(os.listdir(source + '/someone')) * 0.2
for filename in os.listdir(source + '/someone'):
    if count > 0:
        count -= 1
        shutil.move(source + '/someone/' + filename, dest + '/someone/' + filename)
        
        
        
count = len(os.listdir(source + '/everyone')) * 0.2
for filename in os.listdir(source + '/everyone'):
    if count > 0:
        count -= 1
        shutil.move(source + '/everyone/' + filename, dest + '/everyone/' + filename)

In [ ]:
apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
#preparing a Image Data Generator for training
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
#Now I prepare the ImageDataGenerator also for testing and validation
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
#creating datasets objects
#Lo so che le immagini sono di dimensione variabile, ma non so bene come farle passare di qua se no


#manage classes
classes = ['no_one','everyone','someone']


training_dir = cwd + '/Dataset/training'
validation_dir = cwd + '/Dataset/validation'
test_dir = cwd + '/Dataset/test'

#Now I select the data
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               color_mode='rgb',
                                               batch_size=bs,  
                                               classes = classes,
                                               class_mode='categorical',
                                               target_size=(img_h, img_w),
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               color_mode='rgb',
                                               batch_size=bs, 
                                               classes = classes,
                                               class_mode='categorical',
                                               target_size=(img_h, img_w),
                                               shuffle=True,
                                               seed=SEED)

#test_gen = test_data_gen.flow_from_directory(test_dir,
#                                             color_mode='rgb',
#                                             batch_size=bs, 
#                                             class_mode='categorical',
#                                             shuffle=False,
#                                             seed=SEED)
train_gen.class_indices

In [ ]:
#creating datasets objects
#Lo so che le immagini sono di dimensione variabile, ma non so bene come farle passare di qua se no


train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=(tf.TensorShape([None, img_h, img_w, 3]), tf.TensorShape([None, num_classes])))
train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=(tf.TensorShape([None, img_h, img_w, 3]), tf.TensorShape([None, num_classes])))

# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
net = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [ ]:
    finetuning = True

    if finetuning:
        freeze_until = 15 # layer from which we want to fine-tune

        for layer in net.layers[:freeze_until]:
            layer.trainable = False
    else:
        net.trainable = False

    model = tf.keras.Sequential()
    model.add(net)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=500, activation=tf.keras.activations.relu)) #, kernel_regularizer=tf.keras.regularizers.l2(0.001)))
   # model.add(tf.keras.layers.Dense(units=500, activation=tf.keras.activations.relu ))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

    # Visualize created model as a table
    model.summary()

# Visualize initialized weights
#model.weights

In [ ]:

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
callbacks = []
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

#ckpt_dir = os.path.join(cwd, 'ckpts')
#if not os.path.exists(ckpt_dir):
 #   os.makedirs(ckpt_dir)

   # ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   #save_weights_only=True)  # False to save the model directly
#callbacks.append(ckpt_callback)

#model.load_weights(cwd + '/Weights/cp.ckpt')
model.fit(x = train_dataset,
          epochs = 40,  #### set repeat in training dataset
          steps_per_epoch = len(train_gen),
          validation_data = valid_dataset,
          validation_steps = len(valid_gen), 
          callbacks = callbacks)



ahhhhh help
vetter


In [ ]:
from datetime import datetime 
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
image_filenames = next(os.walk('./Dataset/test'))[2]

results = {}

for image_name in image_filenames:
 
   img = Image.open('./Dataset/test/'+image_name).convert('RGB')
   img = img.resize((img_w,img_h))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   
    
   prediction = np.argmax(model.predict(x=img_array/255.))  # predicted class
   

   results[image_name] = prediction

create_csv(results)